# Wiederholung

* ML-Algorithmen eingeführt
    * Adaline, SVM, Entscheidungsbaum, kNN, ... 

* Fallstricke im ML-Prozess besprochen
    * Datenaufteilung
    * Merkmale mit unterschiedlichen Skalen 
    * Überanpassung
    * Hyperparameter der Algorithmen
    * ...




# Datenvorbereitung

Entscheidender Faktor für Erfolg eines Machine Learning Projekts

* __Daten, Daten, Daten__
    * Umfang
    * Auswahl
    * Qualität



Untersuche Daten, __bevor__ ein ML-Algorithmus angewendet wird

* Vorverarbeitung fast immer notwendig
    1. unvollständige (oder falsche) Werte identifizieren
        * evtl. fehlende (oder richtigere) Werte zuweisen
    1. Aufbereitung von kategorialen Daten für den Lernalgorithmus
        * Auswahl der für den Lernalgorithmus maßgeblichen Merkmale
    1. Aufteilung der Datenmenge für Training und Test
    1. Anpassung der Datenmerkmale
    


## Umgang mit fehlenden Daten 
in der Praxis:
* in der Stichprobe oft fehlende Werte 
    * oder falsche Werte
        * Fehler beim Sammeln der Daten 
        * manche Messungen/Merkmale weisen sehr seltsame Werte auf
        * Datenfelder wurden nicht ausgefüllt 
* in den Daten zu erkennen als 
    * NaN (not a number), NULL, "" oder 0 

Beim ignorieren dieser Werte liefern die meisten Tools unvorhersehbare
Ergebnisse



### Fehlende Daten 

Erzeugung eines CSV Datensatzes mit fehlenden Werten

In [ ]:
import pandas as pd
from io import StringIO
import sys

csv_data = '''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

if (sys.version_info < (3, 0)): ## für alte python versionen
    csv_data = unicode(csv_data)
df = pd.read_csv(StringIO(csv_data))
df

* Nach Einlesen in Pandas-Dataframe aus CSV-Format
    * fehlende Werte werden durch NaN ersetzt



Suchen nach fehlenden Werten in großen Datensätzen 

* isnull(): gibt an, wo in einer Dataframe-Tabelle Werte fehlen
* sum(): addiert die fehlenden Werte in einem Dataframe pro Spalte auf

In [ ]:
df.isnull().sum()

Tipp: 
* scikit-learn verwendet im allgemeinen numpy
* aber das Arbeiten mit Pandas-Dataframe ist komfortabler
* das numpy-Array wird über Datafrrame "values" verfügbar 
    * kann so dann an ML-Algorithmen übergeben werden
    * Dokumentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.values.html

In [ ]:
df.values

#### Exemplare und Merkmale entfernen

1. alle Zeilen/Exemplare mit NaN entfernen

In [ ]:
df.dropna(axis=0)

2. alle Spalten mit NaN entfernen

In [ ]:
df.dropna(axis=1)

2. alle Zeilen löschen, in denen alle Spalten NaN enthalten

In [ ]:
df.dropna(how='all')

2. alle Zeilen mit weniger als 4 reellen Zahlen löschen

In [ ]:
df.dropna(thresh=4)

2. alle Zeilen löschen, in denen NaN in einer bestimmten Spalte steht

In [ ]:
df.dropna(subset=['C'])

**Vorteile** 
* sehr einfach 

**Nachteile:**
* Datenmenge wird reduziert 
    * evtl. bis keine vernünftige Analyse mehr möglich ist
* wertvolle Information kann durch Entfernung von Merkmalspalten verloren gehen



### Fehlende Werte ergänzen

Interpolation
* Daten aus den anderen Werten abschätzen 
    * SimpleImputer: https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html?highlight=imputer#sklearn.impute.SimpleImputer
        * Strategien: mean, median, most_frequent, constant
            * most_frequent und constant eignet sich gut für __kategoriale__ Merkmale
    <img  src="./Bilder/BuchKap04/04_01.png" style="width:49%; float:right"/>
    * Teil der **Transformer** Klassen von scikit_learn 
        * wichtigsten Methoden
            * fit: Parameter aus den Daten erlernen 
            * transform: Daten mit den Parametern transformieren 



Mittelwert-Imputation
* den fehlenden Wert einfach durch Mittelwert der gesamten __Merkmalspalte__ ersetzen

In [ ]:
from sklearn.impute import SimpleImputer

imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

#### Vergleich mit Schätzer-API von scikit-learn
<img  src="./Bilder/BuchKap04/04_02.png" style="width:39%; float:right"/>

* Erinnerung: die verwendeten Schätzer zur Klassifizierung 
  * ähnliche Schnittstellen
    * fit
    * predict
    * evtl. auch transform



## Handhabung kategorialer Daten 

### nominale und ordinale Merkmale 

**nominal**: bezeichnende Werte
* Bsp: Farbe von T-Shirts: rot, grün, weiß

**ordinale**: Werte, die geordnet (sortiert) werden können
* Bsp: Größe von T-Shirts: SS < S < M < L < XL



Erstellen einer Beispieldatenmenge:

In [ ]:

df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

Ergebnis: 
* DataFrames mit nominalem Merkmal, ordinalem Merkmal, numerischem Merkmal
* Class-Label kann eine Klassenbezeichung für überwachte Klassifizierung/überwachtes Lernen enthalten 



#### Zuweisung von ordinalen Merkmalen

* Konvertieren der Bezeichungen in ganzzahlige Werte
    * numerische Werte sind immer schneller im Training
* muss von Hand erledigt werden

In [ ]:
df

size_mapping = {'XL': 3,
                'L': 2,
                'M': 1}

df['size'] = df['size'].map(size_mapping)

df

* Rückkonvertierung über Dictionary

In [ ]:
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

#### Codierung der Klassenbezeichnung

Erinnerung: 
* Klassenbezeichungen sollten für die verwendeten Bibliotheken als ganzzahlige Werte vorliegen 
* Einige Toolkits können das selbst machen, 
    * aber es ist guter Stil die Klassenbezeichnungen als Integer-Arrays bereitzustellen 

* Klassenbezeichungen sind nicht geordnet
    * wir können einfach mit 0 anfangen

In [ ]:
import numpy as np

class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

Mapping verwenden

In [ ]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

Rückkonvertierung

In [ ]:
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

Bisher:  alles mit Python-Bordmitteln 
<br>

Jetzt: Verwendet **LabelEncoder** Klasse
* liefert die Label gleich im passenden numpy Array
* und rückwärts

In [ ]:
from sklearn.preprocessing import LabelEncoder

class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y
# rückwärts
class_le.inverse_transform(y)

* fit_transform: getrennter Ablauf von fit und transform 



#### nominale Merkmalen. One-hot-Codierung 

Erinnerung: 
* ordinale Merkmale: Dictionary
* Klassenbezeichung: LabelEncoder

einfacher Ansatz für nominale Merkmale: LabelEncoder

In [ ]:

X = df[['color', 'size', 'price']].values

color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

* Erste Spalte enthält die Farbwerte



**häufigstes Problem** mit kategorialen Daten
* die Werte des Merkmals "color" __erscheinen__ geordnet, obwohl sie es nicht sind
* Algorithmen würden diese Information verwenden, was falsch ist



Lösung: **One-Hot-Codierung** 

* für jeden möglichen Wert wird eine dummy-Merkmalspalte einführt 
    * im Beipiel: drei Merkmale: blue, green, red und binäre Codierung (true oder false)
* verwende One-Hot-Encoder: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html?highlight=onehotencoder#sklearn.preprocessing.OneHotEncoder

In [ ]:
from sklearn.preprocessing import OneHotEncoder

print(df)
colors=['green', 'red', 'blue']

ohe = OneHotEncoder(categories=[colors]  ) # alternative , handle_unknown='ignore'

ohe.fit_transform(df[['color']]).toarray()

* Die Spalten der Farben sind dünnbesetzte Matizen 
    * dünnbesetzte Matrizen können von den Toolkits effizient bearbeitet werden



Alternative: pandas-Bibliothek

In [ ]:
pd.get_dummies(df[['price', 'color', 'size']])

Beachte: 
* es entsteht eine hohe Korrelation zwischen den Merkmalen (hohe Kolinearität)
    * diese kann bei verschiedenen Verfahren problematisch sein
    * kann beim Berechnen von invertierten Matrizen zu numerisch instabilen Abschätzungen führen 
* Reduzierung der Kolinearität durch Entfernen einer Spalte 
    * hier auch wieder mit pandas

In [ ]:
pd.get_dummies(df[['price', 'color', 'size']], drop_first=True)

* Alternative: Verwendung von scikit-learn

In [ ]:
colors=['green', 'red', 'blue']

ohe = OneHotEncoder(categories=[colors], drop='first' )
ohe.fit_transform(df[['color']]).toarray()

Zusammenfassung: 

* Kategoriale Merkmale sind in der __Datenvorbereitung__ mit **viel Sorgfalt** zu behandeln
    * nominale: OneHotCodierung
    * ordinale: Dictionary
    * Klassifikationen: LabelEncoder
* Ziel: Ersetzung von Text durch ganze Zahlen oder binäre Werte 

Jetzt: Datenaufteilung


## Aufteilung einer Datensammlung

Beachte: Die Testdatenmenge dient dem endgültigen und letzten Test des Modells vor dem Praxiseinsatz

* Testdatenmenge muss mit Bedacht gewählt werden

Neue Datenmenge: Weinsammlung
* 178 Datensätze mit 13 Merkmalen 
    * drei Klassen, die für verschiedene Trauben stehen
    * alles Weine aus Italien



In [ ]:

df_wine = pd.read_csv('https://archive.ics.uci.edu/'
                      'ml/machine-learning-databases/wine/wine.data',
                      header=None)

# if the Wine dataset is temporarily unavailable from the
# UCI machine learning repository, un-comment the following line
# of code to load the dataset from a local path:

# df_wine = pd.read_csv('wine.data', header=None)


df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
                   'Proline']

print('Class labels', np.unique(df_wine['Class label']))
df_wine.head()

Suche nach fehlenden Werten

In [ ]:
df_wine.isnull().sum()

Funktion train_test_split

1. Trennung in Merkmale y und Klassenbezeichung X
2. Trennung in Trainings- und Testdaten
    * https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
    * hier 30% der Daten für Testfälle
    * stratify: Häufigkeit der Klassen in Test- und Trainingsdaten identisch

In [ ]:
from sklearn.model_selection import train_test_split

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

X_train, X_test, y_train, y_test =    train_test_split(X, y, 
                     test_size=0.3, 
                     random_state=0, 
                     stratify=y)

Diskussion: 
* durch die Testdaten gehen natürlich Informationen für Training verloren 
    * nicht zu viele Testdaten "verschwenden"
* Abschätzung von Fehlern im Modell wird ungenauer, je kleiner die Testdatenmenge ist
* geeignetes Verhältnis zu finden
    * Üblich: 60:40, 70:30, 80:20 
    * je nach Größe des ursprünglichen Datensatzes
* bei großen Datenmengen ist aber auch 90:10 oder 99:1 vorstellbar. 

* Testdaten können nach der Bewertung des Modells auch nochmals für das Training verwendet werden 
    * das kann aber zu einer Herabsetzung der Verallgemeinerung führen 
        * z.B. bei kleiner Datenmenge und bei Ausreißern in Testdaten
    * danach keine Leistungsbeurteilung mehr möglich, da ja Testdaten fehlen 



## Anpassung der Merkmale

Erinnerung: 
* für verschiedenen Lernalgorithmen war es wichtig, dass die Wertebereiche der Merkmale nicht zu unterschiedlich sind
    * nur bei Entscheidungsbäumen und RandomForest war das kein Problem

* z.B: bei KNN
    * Merkmal A ist im Bereich 1-10 
    * Merkmal B ist im Bereich 1000-10000
    * Der Abstand würde im wesentlichen von B dominiert 



Zwei Ansätze: 
**1.) Normierung**
* Abbildung des Merkmals auf das **Intervall [0,1]**

$$ x_{norm}^{(i)} = \frac{x^{(i)} - x_{min}} {x_{max} - x_{min}}
$$

* in scikit_learn:

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

**2.) Standardisierung**

* für Optimierungen besser geeignet
    * Erinnerung: Gewichte wurden um 0 herum initialisiert
* zentrieren der Werte um 0 mit einer Standardabweichung von 1
* kann mit Ausreißern besser umgehen
    * reagiert weniger auf ungewöhnliche Werte
      
$$ x_{norm}^{(i)} = \frac{x^{(i)} - \mu_x} {\sigma_x}
$$

* Parameter
    * $\mu_x$: Mittelwert einer Merkmalspalte
    * $\sigma_x$: Standardabweichung einer Merkmalspalte 



* in scikit-learn:

In [ ]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

**Wichtig** 
* Standard-Scaler nur einmal an die Trainingdaten anpassen und anwenden
    * fit_transform
* mit den gewonnenen Parametern dann auf die Testdaten anwenden
    * transform



## Auswahl ausagekräftiger Merkmale

Erinnerung: Hinweis für Überanpassung

* Modell funktioniert mit Trainingsdaten deutlich besser als mit Testdaten
    * Verallgemeinerung durch das Modell auf neue Daten gelingt nicht
    * Modell hat eine hohe Varianz
    * Modell ist zu komplex für die vorhandenen Trainingsdaten 



Lösungsansätze: 
1. Mehr Trainingsdaten sammeln 
    * nicht immer machbar
    * wir werden noch Methoden kennenlernen, mit denen man abschätzen kann, ob mehr Daten wirklich helfen

2. Einführung einer Straffunktion zur Regularisierung
    * hatten wir schon mal eingesetzt

1. Auswahl eines einfacheren Modells mit weniger Parametern 

1. Dimensionsreduktion der Daten 



### Regularisierungen - L2 und L1

* Regularisierung: Einführung eines Strafterms, um große Gewichte zu bestrafen 

* aus $J(w)$ wird z.B. $J(w) + \frac{\lambda}{2} \Vert w \Vert^2$

**L2-Norm**: 

$$L2: \Vert w \Vert^2 = \sum_{j=1}^m w_j^2 
$$

* siehe verwendete L2 Regularisierungen 

**L1-Norm**: 

$$L1: \Vert w \Vert = \sum_{j=1}^m \vert w_j\vert 
$$

* statt der Quadrate der Gewichte die Absolutwerte der Gewichtungen 
* beide Methoden können eingesetzt werden. 



### Geometrische Interpretation der Regularisierungen

* im 2D Fall, d.h. es werden 2 Gewichte $w_1$ und $w_2$ gesucht 

**ohne Regularisierung:** 

* Straffunktion basiert auf Summe der quadratischen Abweichungen 
* Straffunktion ist symmetrisch 
* Ziel sind die Werte für $w_1$ und $w_2$ im Minimum der Straffunktion zu finden 



Ohne Regularisierung

<img  src="./Bilder/BuchKap04/04_04.png" style="width:40%; float:right"/>




**L2-Regularisierung:**

<img  src="./Bilder/BuchKap04/04_05.png" style="width:40%; float:right"/>

* Regularisierung ist das Hinzüfugen eines Straftermns
* bei L2 ist das ein Kreis um 0 
    * skaliert wird mit dem Regularisierungsparamter $\lambda$ 

Ziel: Summe aus Straffunktion und Strafterm minimieren 

* Der Punkt an dem die Höhenlinie <br> der <br> Straffunktion den L2 Punkt schneidet ist ideal 
* Hinzufügen des Strafterms sorgt für die <br> Bevorzugung eines einfacheren Modells
    * Varianz des Modells wird verringert




**L1-Regularisierung:**

* Bestimmung der Funktion für $\Vert w \Vert$

$$L1: \Vert w \Vert = \sum_{j=1}^m \vert w_j\vert 
$$

* in zwei Dimensionen

$$\Vert w \Vert = \vert w_0\vert  + \vert w_1\vert \\[5mm]
$$

$$= \begin{cases} 
 - w_0   + \vert w_1\vert ~~, ~~\text{falls}~~ w_0 < 0  \\
 w_0   + \vert w_1\vert ~~, ~~\text{falls}~~ w_0 > 0  
\end{cases}
\\[5mm]
$$

$$= \begin{cases} 
- w_0  - w_1 ~~, ~~\text{falls}~~ w_0 < 0, w_1 < 0  \\
- w_0  + w_1 ~~, ~~\text{falls}~~ w_0 < 0, w_1 > 0  \\
 w_0   - w_1 ~~, ~~\text{falls}~~ w_0 > 0, w_1 < 0 \\
 w_0   + w_1 ~~, ~~\text{falls}~~ w_0 > 0, w_1 > 0 
\end{cases}
$$

* Im Fall der L1-Regularisierung entspricht der Strafterm einem Quadrat



**L1 Regularisierung**

<img  src="./Bilder/BuchKap04/04_06.png" style="width:40%; float:right"/>

Ziel: Summe aus Straffunktion und Strafterm minimieren 
* Der Punkt an dem die Höhenlinie der Straffunktion <br> das L1 Quadrat schneidet ist ideal 
    * im Bild bei $w_1 = 0$ 
* Allgemein: es ist wahrscheinlicher, dass sich die <br> beiden Kurven auf den Koordinatenachsen befinden. 
    * d.h. das die Gewichte teilweise 0 werden 
    * das **begünstigt dünn besetzte Matrizen** 



__Schlussfolgerung__:

* L1-Strafterm begünstigt dünn besetzte Matrizen  
* die meisten Merkmalsgewichte sind null 

* in der Praxis: 
    * nützlich bei hochdimensionalen Datenmengen, in denen es mehrere irrelevante Dimensionen als Objekte gibt
    * **L1-Regularisierung ist ein Verfahren zur Auswahl von Merkmalen**



__Implementierung mit scikit-learn__

* anwenden auf Wine-Daten
* verwende LogisticRegression mit Parameter Penalty='l1'

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1.0, penalty='l1', tol=0.01, solver='saga')
# Note that C=1.0 is the default. You can increase
# or decrease it to make the regulariztion effect
# stronger or weaker, respectively.
lr.fit(X_train_std, y_train)
print('Training accuracy:', lr.score(X_train_std, y_train))
print('Test accuracy:', lr.score(X_test_std, y_test))

Diskussion
* Sowohl die Trainings als auch die Testdaten werden korrekt vorhergesagt

* Es werden drei Klassen in einem One-Vs-Rest Ansatz angepasst. 
    * Klasse 1: positiv und Klasse 2 und 3 sind negativ Auswahl
    * Klasse 2: positiv und Klasse 1 und 3 sind negativ Auswahl
    * Klasse 3: positiv und Klasse 1 und 2 sind negativ Auswahl

* Das Modell enthält drei Werte für $w_0$

In [ ]:
lr.intercept_

Die Werte der einzelnen Gewichte für die drei Klassen

In [ ]:
np.set_printoptions(3)
lr.coef_[lr.coef_!=0].shape
lr.coef_

* Jede Zeile besteht aus 13 Gewichten

* Einige der Gewichte sind null., d.h. das sind eher irrelevante Werte.
* Aber einige der Gewichte sind nicht null. 

* eine dünne Besetzung läßt sich erzwingen durch Verändern des Parameters "C"

* Plotten der Gewichtungsfaktoren als Funktion von C



In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = plt.subplot(111)
    
colors = ['blue', 'green', 'red', 'cyan', 
          'magenta', 'yellow', 'black', 
          'pink', 'lightgreen', 'lightblue', 
          'gray', 'indigo', 'orange']

weights, params = [], []
for c in np.arange(-4., 6.):
    lr = LogisticRegression(penalty='l1', C=10.**c, random_state=0, solver='saga')
    lr.fit(X_train_std, y_train)
    weights.append(lr.coef_[1])
    params.append(10**c)

weights = np.array(weights)

for column, color in zip(range(weights.shape[1]), colors):
    plt.plot(params, weights[:, column],
             label=df_wine.columns[column + 1],
             color=color)
plt.axhline(0, color='black', linestyle='--', linewidth=3)
plt.xlim([10**(-5), 10**5])
plt.ylabel('weight coefficient')
plt.xlabel('C')
plt.xscale('log')
plt.legend(loc='upper left')
ax.legend(loc='upper center', 
          bbox_to_anchor=(1.38, 1.03),
          ncol=1, fancybox=True)
plt.show()

* mit dem Parameter C kann man den Einfluss der verschiedenen Gewichte/Merkmale "regulieren" 
* Dimensionsreduzierung ist möglich



### Algorithmen zur sequentiellen Auswahl von Merkmalen 

*  Verfahren zur **Dimensionsreduzierung**
    * auch nützlich bei nicht regularisierbaren Modellen

__Dimensionsreduzierung__
* Unterteilung in zwei Hauptkategorien 
    * Auswahl von Merkmalen
        * Verwenden einer Untermenge von Merkmalen
        * ($\rightarrow$ gleich)
    * Extrahierung von Merkmalen
        * Konstruktion eines neuen Merkmalraums durch Herleiten von Informationen aus der Menge der Merkmale
       * ($\rightarrow$ später)



**SBS Sequenticial Backward Selection (sequenzielle Rückwärtsauswahl)** 

* klassischer Algorithmus zur Auswahl von Merkmalen 
* Erhöhung der Effizienz der Berechnung durch Dimensionsreduktion bei minimaler Leistungseinbuße des Klassifizierers
* gehört zur Familie der "greedy"-Algorithmen
    * bei jeder Stufe einer kombinatorischen Suchaufgabe wird eine lokale optimale Auswahl getroffen 
        * muß nicht zur global optimalen Auswahl führen 
        * optimale Auswahl wäre mit viel größerem Rechenaufwand verbunden



Ziel: 
* den anfänglichen $d$-dimensionalen Raum auf einen $k$-dimensionalen Raum reduzieren ($k<d$)
* automatische Auswahl der Untermenge der Merkmale mit der größten Bedeutung 
* oder Verallgemeinerungsfehler des Modells verringern durch Entfernen irrelevanter Merkmale
* falls ein Objekt an Überanpassung leidet, kann die Dimensionsreduzierung evtl. sogar die Vorhersagekraft verbessern 



**Idee SBS**

* entferne der Reihe nach Merkmale
    * bis zur erwünschten Anzahl von Merkmalen 
* Zur Bestimmung des auf jeder Stufe zu entfernenden Merkmals wird eine zu minimierende Straffunktion $J$ definiert
    * z.B. die Differenz der Leistung des Klassifizieres vor und nach dem Entfernen bestimmter Merkmale
    * das zu entfernende Merkmal wäre dasjenige, das den günstigsten Wert für das Entscheidungskriterium liefert 
    * oder: das Merkmal, das zu der geringsten Leistungeinbuße führt 
  


**Definition des SBS Algorithmus** 

1. Initialisiere den Algorithmus mit $k=d$ mit $d$ als Dimension des vollständigen Merkmalsraums $X_d$

1. Festlegen des Merkmals $x^-$ zur Maximierung des Kriteriums $x^- = argmaxJ (X_k, - x)$ wobei $x \in X_k$ 

1. Entfernen des Merkmals $x^-$ aus der Merkmalsmenge: $X_{k-1} := X_k - x^- ; k := k -1$

1. Beenden, wenn $k$ gleich der Anzahl der erwünschten Merkmale ist, ansonsten mit Schritt 2 fortfahren 

Anwendung: 
* SBS ist nicht in scikit-learn implementiert
* eigene Implementierung notwendig



Implementierung
* k_features: Anzahl der erwünschten Merkmale
* accuray_score von scitkit-learn schätzt die Leistungsfähigkeit ab
* fit Methode
    * erzeugt jedes Mal aus der ursprünglichen Menge der Daten eigene Trainings- und Testdaten 
    * innerhalb der while Schleife werden mit itertools.combination die Merkmalsuntermengen erstellt und bewertet
    *

In [ ]:
from sklearn.base import clone
from itertools import combinations
from sklearn.metrics import accuracy_score

class SBS():
    def __init__(self, estimator, k_features, scoring=accuracy_score,
                 test_size=0.25, random_state=1):
        self.scoring = scoring
        self.estimator = clone(estimator)
        self.k_features = k_features
        self.test_size = test_size
        self.random_state = random_state

    def fit(self, X, y):
        
        X_train, X_test, y_train, y_test =             train_test_split(X, y, test_size=self.test_size,
                             random_state=self.random_state)

        dim = X_train.shape[1]
        self.indices_ = tuple(range(dim))
        self.subsets_ = [self.indices_]
        score = self._calc_score(X_train, y_train, 
                                 X_test, y_test, self.indices_)
        self.scores_ = [score]

        while dim > self.k_features:
            scores = []
            subsets = []

            for p in combinations(self.indices_, r=dim - 1):
                score = self._calc_score(X_train, y_train, 
                                         X_test, y_test, p)
                scores.append(score)
                subsets.append(p)

            best = np.argmax(scores)
            self.indices_ = subsets[best]
            self.subsets_.append(self.indices_)
            dim -= 1

            self.scores_.append(scores[best])
        self.k_score_ = self.scores_[-1]

        return self

    def transform(self, X):
        return X[:, self.indices_]

    def _calc_score(self, X_train, y_train, X_test, y_test, indices):
        self.estimator.fit(X_train[:, indices], y_train)
        y_pred = self.estimator.predict(X_test[:, indices])
        score = self.scoring(y_test, y_pred)
        return score

Ausführen der Implementierung
* es wird ein KNN-Algorithmus verwendet

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
# selecting features
sbs = SBS(knn, k_features=1)
sbs.fit(X_train_std, y_train)

* und darstellen der Ergebnisse

In [ ]:
# plotting performance of feature subsets
k_feat = [len(k) for k in sbs.subsets_]

plt.plot(k_feat, sbs.scores_, marker='o')
plt.ylim([0.7, 1.02])
plt.ylabel('Accuracy')
plt.xlabel('Number of features')
plt.grid()
plt.tight_layout()
plt.show()

Diskussion
* Die Klassifizierungsrate bei 13 Merkmalen ist nicht die Beste
* das Entfernen von Merkmalen kann die Klassifizierungsrate verbessern (12, 11, 10, 9, 8, 7, 3)
* Selbst bei 3 Merkmalen werden 100% Klassifizierung erreicht 

Welche drei Merkmale machen nun einen solch gute Klassifizierung?

In [ ]:
k3 = list(sbs.subsets_[10])
print(df_wine.columns[1:][k3])

Vergleiche das ursprünglichen Modells mit dem reduzierten Modell 

* Ursprüngliche KNN Klassifizierer

In [ ]:
knn.fit(X_train_std, y_train)
print('Training accuracy:', knn.score(X_train_std, y_train))
print('Test accuracy:', knn.score(X_test_std, y_test))

* KNN Klassifizierer mit den drei Merkmalen

In [ ]:
knn.fit(X_train_std[:, k3], y_train)
print('Training accuracy:', knn.score(X_train_std[:, k3], y_train))
print('Test accuracy:', knn.score(X_test_std[:, k3], y_test))

* Nutzung von weniger als einem Viertel der Merkmale verschlechtert die Vorhersage nur geringfügig
    * die drei liefern weniger Vorhersagekraft als die 13
    * bei größeren Datenmengen kann das wieder besser werden 

Vorteile: 
* die Größe der Datenmenge wurde reduziert
    * Datenbeschaffung in der Praxis wird günstiger
    * einfachere Modelle, die leichter zu interpretieren sind 
    * Berechung der Vorhersagen wird effizienter



## Bedeutung von Merkmalen in RandomForest

Bisher: 
* L1-Regression zur Erkennung irrelevante Merkmale 
* SBS-Algorithmus zur Auswahl relevanter Merkmale 

Jetzt: 
* Erkennen relevanter Merkmale mit RandomForest



Implementierung
* RandomForest ist eine EnsembleMethode
* scikit-learn hat eine Implementierung RandomForestClassifier 
    * diese liefert das Attribute *feature_importance* 

Anwendung: 
* wine-Daten
* trainiere RandomForest mit 500 Bäumen 
* erstelle eine nach Bedeutung sortierte Liste 
* beachte: Es müssen keine normierten oder standardisierten Werte verwendet werden 



In [ ]:
from sklearn.ensemble import RandomForestClassifier

feat_labels = df_wine.columns[1:]

forest = RandomForestClassifier(n_estimators=500,
                                random_state=1)

forest.fit(X_train, y_train)
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))

plt.title('Feature Importance')
plt.bar(range(X_train.shape[1]), 
        importances[indices],
        align='center')

plt.xticks(range(X_train.shape[1]), 
           feat_labels[indices], rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()
plt.show()

* die Ausgabewerte sind normiert und ergeben in der Summe 1
* im Ergebnis sind zwei der drei wichtigsten Merkmale auch bei SBS bestimmt worden 

Achtung: 
* bei stark korrellierten Merkmalen kann die Interpretierbarkeit leiden 
    * eine der korrelierten Merkmale kann oben stehen, das andere gar nicht erfasst sein



Verwenden der Reihenfolge in scikit-learn
* SelectFromModel
    * Wählt aus dem ermittelten Modell die Merkmale über einer Schwelle aus
    * im Folgenden können dann nur diese Merkmale verwendet werden

Anwendung

In [ ]:
from sklearn.feature_selection import SelectFromModel

sfm = SelectFromModel(forest, threshold=0.1, prefit=True)
X_selected = sfm.transform(X_train)
print('Number of features that meet this threshold criterion:', 
      X_selected.shape[1])

for f in range(X_selected.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))

## Summary

* Datenvorbereitung ist wesentlich für Machine Learning

* Umgang mit fehlenden Daten 

* Umgang mit kategorialen Daten sehr wichtig
    * Unterscheidung nominaler und ordinaler Merkmale 

* Erste Methoden zur Merkmalsreduzierung 
    * L1-Regularisierung
    * SBS-Algorithmus
    * RandomForest
    * SelectFromModel

* Ziel ist die Reduzierung der Komplexität von Modellen bzw. die Verhinderung von Überanpassung